## ML Project - Dataset 1 - Spotify

Exploration

In [1]:
import numpy as np
import pandas as pd
from scipy import stats as sps
from matplotlib import pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
from IPython.display import display, Latex

file_path = '/Users/dakshkhetarpaul/Desktop/ML443-Final-Project/Datasets/Spotify Most Streamed Songs.csv'

# Load the data
training_data = pd.read_csv(file_path)